In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
#import shap
from sklearn.pipeline import Pipeline

In [2]:
#from sklearnex import patch_sklearn
#patch_sklearn()

In [12]:
dataset = pd.read_csv("datasets.csv")  # should I use all rows?

#testd = pd.read_csv("redox_test.csv")  # this is the test file
dataset
type(dataset)

pandas.core.frame.DataFrame

In [13]:
# finger
# X2=pd.concat([finger,data],axis=1)  # FP's first
dataset.drop(['ID','redoxpot2','SMILES'], axis = 1, inplace=True) # Id, column SMILES and redoxpot2 can be dropped
label ='redoxpot1' # focus on prediction of redoxpot1, no need to predict redoxpot2

In [15]:
type(dataset)
dataset

,HomoA,LumoA,GapA,HomoAH,LumoAH,GapAH,HomoAH2,LumoAH2,GapAH2,redoxpot1
0,-7.9848,-3.6582,4.3265,-7.5462,-3.5189,4.0272,-6.0675,-0.4076,5.6599,0.9121
1,-6.6740,-3.3543,3.3197,-6.6104,-3.0729,3.5374,-5.8577,-0.2250,5.6327,0.8415
2,-6.3660,-3.3184,3.0476,-6.2117,-3.0280,3.1837,-5.9100,-0.3589,5.5510,0.8053
3,-7.1440,-3.6882,3.4558,-6.8373,-3.5720,3.2653,-5.9666,-0.4971,5.4694,0.9603
4,-7.5521,-3.5521,4.0000,-7.2218,-3.3578,3.8640,-5.9178,-0.1763,5.7415,0.8734
...,...,...,...,...,...,...,...,...,...,...
1984,-6.9374,-3.0191,3.9184,-6.7611,-3.0876,3.6735,-5.0583,-1.6841,3.3742,0.2191
1985,-7.0019,-3.0019,4.0000,-6.7845,-3.0294,3.7551,-5.0427,-1.6958,3.3470,0.1941
1986,-6.9309,-3.0125,3.9184,-6.7957,-3.0678,3.7279,-5.0055,-1.6313,3.3742,0.3965
1987,-6.8912,-3.0000,3.8912,-6.6754,-3.0291,3.6463,-5.0566,-1.7641,3.2925,0.1937


In [16]:
#testd= testd[['Atomic number of central metal-cat', '4s electrom', '3d electron', 'no.atom-complex', 'no.ligand', 'no.N','no.O','no.C','no.BONDS','no.ring each ligand']]
data = dataset.sample(frac=1)   # randomize the data order
y_all = data[label]
train_data, test_data = train_test_split(data, test_size=0.25)

# train_data, remaining_data = train_test_split(data, test_size=0.5)
# val_data, test_data = train_test_split(remaining_data, test_size=0.25)

In [17]:
y_all
#testd
#data

0       0.9121
1119    0.5062
1410    0.2140
1857    0.3948
1465    0.2653
         ...  
1531    0.5915
573     0.5571
1829    0.3331
541     1.2960
546     0.9438
Name: redoxpot1, Length: 1989, dtype: float64

In [22]:
X = train_data.drop(columns= [label]) # .to_numpy()
y = train_data[label]

X_test = test_data.drop(columns= [label]) # .to_numpy()
y_test = test_data[label]

#unscale_tX = test_X

# scaler = MinMaxScaler()
# scaler = scaler.fit(X)

# X = scaler.transform(X)
#print(X)
# test_X = scaler.transform(test_X)
#print(unscale_tX)
#print(test_X)

#print(train_data.shape)

In [23]:
# model = GradientBoostingRegressor()
#model = RandomForestRegressor()
#krr = KernelRidge()
#krr = RandomForestRegressor()
#scaler = MinMaxScaler()
#model = Pipeline([('scaler', scaler), ('krr', krr)])
##
#model = RandomForestRegressor()
#model = GradientBoostingRegressor()

###

RF="KR"

if RF=="GB":
#  # GB method next 2 lines
  model = GradientBoostingRegressor()
  parameters = {'learning_rate': np.arange(0.05, 0.3, 0.05), "loss": ['huber'],"n_estimators": range(20, 80, 10),
                'subsample': [1.0, 0.9]}   # for GB
#parameters = {'learning_rate': np.arange(0.05, 0.3, 0.05), "loss": ['ls', 'huber'], "n_estimators": range(20, 80, 10),
#               'subsample': [1.0, 0.9]}   # for GB
if RF=="KR":
  model = KernelRidge()
  parameters = {"alpha": np.arange(0.5, 3.5, 0.2), "kernel": ['linear', 'polynomial', 'laplacian'], "degree": [2,3]}  #  KRR

if RF=="RF":
# RF methods next 2 lines
  model = RandomForestRegressor()
  parameters = {"n_estimators": range(110, 190, 20), "min_samples_split":[2]}  # for RF

                                                                      
#parameters = {"krr__alpha": np.arange(0.5, 3.5, 0.2), "krr__kernel": ['linear', 'polynomial', 'laplacian'], "krr__degree": [2,3]}
###
clf = GridSearchCV(model, parameters, cv = 10, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')

In [24]:
clf.fit(X,y)

Fitting 10 folds for each of 90 candidates, totalling 900 fits
[CV] END .................alpha=0.5, degree=2, kernel=linear; total time=   0.2s
[CV] END .................alpha=0.5, degree=2, kernel=linear; total time=   0.4s
[CV] END .................alpha=0.5, degree=2, kernel=linear; total time=   0.3s
[CV] END .............alpha=0.5, degree=2, kernel=polynomial; total time=   0.3s
[CV] END .............alpha=0.5, degree=2, kernel=polynomial; total time=   0.4s
[CV] END .............alpha=0.5, degree=2, kernel=polynomial; total time=   0.4s
[CV] END ..............alpha=0.5, degree=2, kernel=laplacian; total time=   0.5s
[CV] END ..............alpha=0.5, degree=2, kernel=laplacian; total time=   0.4s
[CV] END ..............alpha=0.5, degree=2, kernel=laplacian; total time=   0.4s
[CV] END .................alpha=0.5, degree=3, kernel=linear; total time=   0.3s
[CV] END .................alpha=0.5, degree=3, kernel=linear; total time=   0.3s
[CV] END .................alpha=0.5, degree=3,

GridSearchCV(cv=10, estimator=KernelRidge(), n_jobs=-1,
             param_grid={'alpha': array([0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9,
       3.1, 3.3]),
                         'degree': [2, 3],
                         'kernel': ['linear', 'polynomial', 'laplacian']},
             scoring='neg_mean_squared_error', verbose=2)

In [26]:
clf.best_params_

{'alpha': 0.5, 'degree': 2, 'kernel': 'laplacian'}

In [27]:
model = clf.best_estimator_
model

KernelRidge(alpha=0.5, degree=2, kernel='laplacian')

In [ ]:
#for param in pd.DataFrame(clf.cv_results_).sort_values("rank_test_score")["params"][0:5]:
#    print(param)

In [28]:
result_df = pd.DataFrame(clf.cv_results_)
result_df = result_df.sort_values("rank_test_score")
for i, row in result_df.iterrows():
    print(row["mean_test_score"], row["params"])

-0.021117397682291355 {'alpha': 0.5, 'degree': 2, 'kernel': 'laplacian'}
-0.021117397682291355 {'alpha': 0.5, 'degree': 3, 'kernel': 'laplacian'}
-0.022223190936258137 {'alpha': 0.7, 'degree': 2, 'kernel': 'laplacian'}
-0.022223190936258137 {'alpha': 0.7, 'degree': 3, 'kernel': 'laplacian'}
-0.023232188101495837 {'alpha': 0.8999999999999999, 'degree': 3, 'kernel': 'laplacian'}
-0.023232188101495837 {'alpha': 0.8999999999999999, 'degree': 2, 'kernel': 'laplacian'}
-0.02416141420973852 {'alpha': 1.0999999999999999, 'degree': 3, 'kernel': 'laplacian'}
-0.02416141420973852 {'alpha': 1.0999999999999999, 'degree': 2, 'kernel': 'laplacian'}
-0.025024205219597064 {'alpha': 1.2999999999999998, 'degree': 3, 'kernel': 'laplacian'}
-0.025024205219597064 {'alpha': 1.2999999999999998, 'degree': 2, 'kernel': 'laplacian'}
-0.0258306597765715 {'alpha': 1.4999999999999998, 'degree': 3, 'kernel': 'laplacian'}
-0.0258306597765715 {'alpha': 1.4999999999999998, 'degree': 2, 'kernel': 'laplacian'}
-0.0265885

# SHAP analysis

In [ ]:
#all = FALSE
#if all:
# for all data 
#X2=data.drop(columns=[Target])  # this have the headers, all randomized data
#y_all = data[Target]
#smodel=model.fit(X2,y_all)      # rather slow
#explainer = shap.Explainer(smodel)
#shap_values = explainer(X2)
#  for train data 
print('===== SHAP FOR TRAIN DATA ====')
print('===== NOTE SHAP ANALYSIS DOES NOT WORK WITH KR METHOD ====')
smodel=model.fit(X,y)      # rather slow
explainer = shap.Explainer(smodel)
shap_values = explainer(X)
print('===== SHAP DONE ====')

In [ ]:
features = X.columns
#print('Features ', features[:])
featureNames=[]
for i in range(0,len(features)-1): # ignore the first and last feature 
    featureNames.append(features[i])
nFeatures=len(featureNames)
print('Feature Names ', featureNames)
#print('Feature 65 ', features[65])
#print('Feature 70 ', features[70])

#shapreg=model     # the used model, here RF
Xs = X  # data2   # this have the headers SHAP
print('\n shap base_values ',shap_values.base_values[0])
base=shap_values.base_values[0]
shap.plots.beeswarm(shap_values,max_display=15) # ,feature_names=featureNames)
shap.plots.bar(shap_values) # ,feature_names=featureNames) # all shap values
#shap.plots.waterfall(shap_values[11])
#shap.waterfall_plot(base,shap_values[11],X[11])
#for i in range(3):
#   shap.plots.bar(shap_values[i],show=False) # all shap values 
#   plt.title(metals.at[i,'name'])
#   plt.show()

In [ ]:
print(' =========    SHAP 2 ANALYSIS START ========')
shap2_values = shap.TreeExplainer(smodel).shap_values(X)  # this is slow  X for train data, X2 for all 
print(' =========    SHAP 2 ANALYSIS DONE ========')

In [ ]:

F1="HomoAH2"
F2="LumoA"
F3="24"
F4="196"
print(' =========    SHAP DEPENDENCE  =============')
shap.dependence_plot(F1, shap2_values, X)
shap.dependence_plot(F2, shap2_values, X)
shap.dependence_plot(F3, shap2_values, X)
shap.dependence_plot(F4, shap2_values, X)

In [ ]:

shapint_values=shap.TreeExplainer(smodel).shap_interaction_values(X)  # very slow
print(' =========    SHAP INT ANALYSIS  =============')

In [ ]:
shap.dependence_plot((F1, F2), shapint_values, X)
shap.dependence_plot((F1, F3), shapint_values, X)

In [ ]:
pred_y = model.predict(X)
#print(' shape y',np.shape(y),'shape X',np.shape(X))
yy=[]   # y is suffled  yy contain the unsuffled data
i=0
for yp in y:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1
print('y(0:18) \n',y[0:8],yy[0:8])
print(' shape y',np.shape(y),'shape X',np.shape(X))

cut=0.25
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(y)):
    if abs(pred_y[ii]-yy[ii]) > cut:
#      print('Xii, i',ii,X[ii,:])
#       Xtest=X[ii,:].reshape(1, -1) 
#       model.predict(Xtest)
      print('Pred E ',pred_y[ii],' E_DFT',yy[ii],'diff',abs(pred_y[ii]-yy[ii]))

print('---------------------------------------------------------------')
#Xtest=testd.to_numpy()
#for i in range(len(Xtest)):
#  Xtt=Xtest[i,:].reshape(1, -1)
#  print('test ',i,Xtest[i,:])
#  print('Test E',float(model.predict(Xtt)))

plt.plot(pred_y, y, 'o', label = "ML")
plt.plot(y, y)
plt.xlabel("Predicted Energy (eV)")
plt.ylabel("DFT energy (eV)")
plt.legend()
r_sqrt = r2_score(y, pred_y)
print("\n R^2 on ML training set:", r_sqrt)
#print(r"R^2 on BEP training set:", r_BEP)

In [ ]:
shapasum=[]
shapy=[]
#shapytest=0.0 
for i in range(len(y)):
    ys=float(base+sum(shap_values.values[i]))
    shapy.append(ys)
#    shapytest=shapytest+abs(ys-y[i])
#    print(i,'SHAP values ',shap_values.values[i],' SHAP predic',ys)                

plt.plot(shapy, y, 'o', label = "Shap")
plt.plot(y, y)
plt.xlabel("Shap predicted Energy (eV)")
plt.ylabel("DFT energy (eV)")
plt.legend()
plt.show()
r_sqrt = r2_score(shapy, y)
print("R^2 on Shap fit DFT set:", r_sqrt)
#plt.plot(shapy, pred_y, 'o', label = "Shap")
#plt.plot(y, y)
#plt.xlabel("Shap predicted Energy (eV)")
#plt.ylabel("ML predic (eV)")
#plt.legend()
r_sqrt = r2_score(shapy, pred_y)
print("R^2 on Shap fit ML set:", r_sqrt)


In [ ]:
#test_X = test_data.drop(columns= ['dEf', 'name']).to_numpy()
#test_y = test_data['dEf']

print(' shape y',np.shape(test_y),'shape X',np.shape(test_X))
predt_y = model.predict(test_X)

yy=[]   # test_y is suffled  yy is the unsuffled data
i=0
for yp in test_y:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1

cut=0.3
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(test_y)):
    if abs(predt_y[ii]-yy[ii]) > cut:
#      print('test_X, i',ii,test_X[ii,:])
#       Xtest=X[ii,:].reshape(1, -1) 
      print('Pred E ',predt_y[ii],' E_DFT',yy[ii],'diff',abs(predt_y[ii]-yy[ii]))
print('-----------------------------------------------------------------')

plt.plot(predt_y, test_y, 'o', label = "ML")
plt.plot(test_y, test_y)
plt.xlabel("Predicted redox (eV)")
plt.ylabel("DFT redox (eV)")
plt.legend()
plt.show()
r_sqrt = r2_score(test_y, predt_y)
print("R^2 on TEST set:", r_sqrt)


In [ ]:
feature_importance = model.feature_importances_
data_used = train_data.drop(columns= [Target])
plt.bar(data_used.columns, feature_importance)

In [ ]:
#print(feature_importance)